In [37]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

# активационная функция
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# производная активационной функции
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

# квадратичная целевая функция
# test_data - кортеж из входных значений и списка значений классов
def cost_function(network, test_data):
    c = 0
    for example, y in test_data:
        c += np.sum((network.feedforward(example) - y) ** 2)
    return c /(2 * len(test_data))


In [38]:
class Network:
    def __init__(self, shape, activation_function, activation_function_derivative, debug= True):
        self.shape = shape
        self.biases = [np.random.randn(y, 1) for y in shape[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(shape[:-1], shape[1:])]
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
        self.debug = debug

    # прогнать до конца примеры из input_matrix
    def feedforward(self, input_matrix):
        for b, w in zip(self.biases, self.weights):
            # weigts - (массив матриц)
            input_matrix = self.activation_function(np.dot(w, input_matrix) + b)
        return input_matrix

    # обновление параметров нейронной сети (веса, смещения), сделав шаг градиентного спуска
    # на основе алгоритма обратного распространения ошибки
    # alpha - learning rate
    def back_prop_step(self, data, alpha):
        # значения dJ/db для каждого слоя
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        # значения dJ/dw (ошибки) для каждого слоя
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # для каждого примера из батча применяем бек пропогейшн
        for x, y in data:
            delta_nabla_b, delta_nabla_w = self.back_prop_single_example(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        eps = alpha / len(data)

        # обновляем параметры сети
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]

    def prepare_data(data, classes_count):
        return [(row[:-classes_count].reshape(-1,1), row[-classes_count:].reshape(-1,1)) for row in data]

    # алгоритм градиентного спуска
    def SGD(self, data_in, epochs, alpha, classes_count):
        prepared_data = Network.prepare_data(data_in, classes_count)
        errors = []
        
        for epoch in range(epochs):
            self.back_prop_step(prepared_data, alpha)
            error = cost_function(self, prepared_data)
            if self.debug:
                print(f'epoch: {epoch} -  error:{error}')
            errors.append(error)
            

        return errors
    
    # возвращает вектор частных производных квадратичной целевой функции по активациям выходного слоя
    def cost_derivative(self, output_activations, y):
        return output_activations - y

    # алгоритм обратного распространения ошибки для одного примера из тренировочной выборки
    # возвращает кортеж (nabla_b, nabla_w) - градиентов по слоям по смещениям и весам соответственно
    def back_prop_single_example(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)
        # массив векторов активаций нейронов
        activations = [x]
        # массив векторов сумматорных функций от активаций предыдущих слоев
        summatories = []
        # b - вектор смещений нейронов данного слоя
        # w - матрица весов, входящих в данный слой
        for b, w in zip(self.biases, self.weights):
            summatories.append(np.dot(w, activations[-1]) + b)
            activation = self.activation_function(summatories[-1])
            activations.append(activation)

        # обратное распространение (backward pass)
        
        # ошибка для выходного слоя
        delta = self.cost_derivative(activations[-1], y) * self.activation_function_derivative(summatories[-1])
        # производная J по биасам выходного слоя
        nabla_b[-1] = delta
        # производная J по весам выходного слоя
        nabla_w[-1] = delta.dot(activations[-2].T)

        # Здесь l = 1 означает последний слой, l = 2 - предпоследний и так далее.  
        for l in range(2, len(self.shape)):
            derivative = self.activation_function_derivative(summatories[-l])
            # ошибка на слое L-l
            delta = derivative * self.weights[-l + 1].T.dot(delta)
            # производная J по смещениям L-l-го слоя
            nabla_b[-l] = delta
            # производная J по весам L-l-го слоя
            nabla_w[-l] = delta.dot(activations[-l - 1].T) 
        return nabla_b, nabla_w

In [39]:
def draw_class(index, normalized_data, class_count):
    class_n = [float(network.feedforward(row[:-class_count].reshape(-1,1))[index]) for row in normalized_data]
    class_real = [float(row[-class_count:].reshape(-1,1)[index]) for row in normalized_data]

    plt.scatter(range(len(class_real)), class_real)
    plt.plot(class_n,'g')
    return class_n

def rmse(network, train_data):
    print(cost_function(network, train_data))
    return np.sqrt(cost_function(network, train_data))

def get_normalized_data(url, classes_count):
    data = pd.read_csv(url, delimiter=';',encoding='utf-8')
    normalized = (data / data.max())
    normalized.iloc[:, :-classes_count] = normalized.iloc[:, :-classes_count].fillna(0)
    mean = normalized.iloc[:, -classes_count:].mean()
    normalized.iloc[:, -classes_count:] = normalized.iloc[:, -classes_count:].fillna(mean)
    return normalized       

In [40]:
normalized_train_data = get_normalized_data('https://raw.githubusercontent.com/SmirAlex/back-propogation/master/train_data.csv', 2)
normalized_test_data = get_normalized_data('https://raw.githubusercontent.com/SmirAlex/back-propogation/master/test_data.csv', 2)

In [41]:
pd.read_csv('https://raw.githubusercontent.com/SmirAlex/back-propogation/master/train_data.csv', delimiter=';')


,Рзаб,Pлин,Руст.1,Рзаб.1,Рлин,Туст,Тна шлейфе,Тзаб,Tлин,Дебит ст. конд.,Дебит воды,Дебит смеси,Дебит гааз,Дебит кон нестабильный,Дебит воды.1,Pсб.1,G_total,КГФ
0,370.1,101.800000,249.00000,359.6,101.800000,53.0,31.7,103.200,32.500000,83.600000,0.400000,231.248840,1610.368866,131.3,0.4,92.376018,2.782623,311.9094
1,364.6,101.300000,231.00000,338.1,102.400000,58.8,37.6,103.000,38.600000,104.700000,1.400000,309.001660,2310.226429,158.5,1.4,91.685171,3.697781,288.6003
2,357.1,101.600000,211.00000,314.8,100.600000,63.6,42.8,102.600,43.400000,114.300000,1.900000,388.114980,3039.489680,172.3,1.9,90.204787,4.515073,248.7906
3,347.4,98.400000,187.00000,291.5,99.000000,64.7,46.2,102.000,46.100000,121.300000,3.300000,455.214520,3824.082686,181.5,3.3,88.033555,5.217673,223.5591
4,337.7,99.200000,169.00000,270.3,99.500000,64.0,49.4,104.400,49.900000,129.800000,4.600000,504.587780,4299.104676,190.6,4.7,88.527017,5.765092,215.1486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,318.0,97.270564,205.00000,312.5,95.523484,66.0,46.9,104.643,46.715433,102.942937,3.226368,625.486783,5452.400000,161.1,3.3,87.737478,NaN,132.0000
61,316.1,101.819504,219.87525,316.2,101.922258,24.2,NaN,106.200,23.662576,39.741318,0.600000,136.815245,884.000000,68.9,0.5,100.468789,NaN,241.0000
62,281.5,103.865199,201.63675,275.9,103.971613,35.5,NaN,106.400,36.558231,67.582907,1.398022,259.198290,1867.000000,120.4,1.4,102.245250,NaN,216.0000
63,257.9,103.064884,182.38500,258.0,103.451290,42.3,NaN,106.470,42.632478,71.294036,2.183448,311.236191,2445.000000,123.9,2.3,101.258327,NaN,191.0000


In [42]:
normalized_test_data

,Рзаб,Pлин,Руст.1,Рзаб.1,Рлин,Туст,Тна шлейфе,Тзаб,Tлин,Дебит ст. конд.,Дебит воды,Дебит смеси,Дебит гааз,Дебит кон нестабильный,Дебит воды.1,Pсб.1,G_total,КГФ
0,0.578301,0.987040,0.519298,0.582982,1.000000,0.722857,0.000000,1.000000,0.936584,0.462796,0.398471,0.528347,0.542822,0.459327,0.400000,1.000000,NaN,0.732143
1,0.798850,0.952741,0.744521,0.805058,0.952953,0.571429,0.501931,0.998126,0.479630,0.287342,0.093750,0.250735,0.203733,0.298903,0.076923,0.959615,NaN,0.977679
2,0.780889,0.950851,0.723744,0.786881,0.961556,0.685714,0.536680,0.999063,0.527778,0.334177,0.156250,0.293408,0.244375,0.352251,0.153846,0.959615,NaN,0.973214
3,0.727399,0.950851,0.668338,0.732877,0.961556,0.771429,0.677606,0.999063,0.672222,0.404430,0.234375,0.385262,0.348160,0.419221,0.246154,0.962500,NaN,0.892857
4,0.669830,0.956522,0.599080,0.674921,0.967291,0.771429,0.789575,0.998126,0.759259,0.398734,0.265625,0.465368,0.443921,0.437003,0.292308,0.962500,NaN,0.718750
5,0.612261,0.957467,0.533285,0.616965,0.971114,0.800000,0.880309,0.997188,0.855556,0.436709,0.140625,0.523675,0.530787,0.428301,0.153846,0.964423,NaN,0.700893
6,0.567059,0.958412,0.477879,0.571128,0.957733,0.814286,0.951737,0.997470,0.920370,0.450000,0.125000,0.547578,0.579103,0.441922,0.123077,0.968269,NaN,0.687500
7,0.454902,0.870063,0.564451,0.458377,0.869271,0.628571,0.494208,0.961106,0.480422,0.214810,0.037382,0.224913,0.207047,0.230420,0.030769,0.865385,NaN,0.803571
8,0.597281,0.875122,0.567914,0.603530,0.885492,0.600000,0.478764,0.960703,0.477919,0.212847,0.037030,0.224864,0.203733,0.227393,0.030769,0.869231,NaN,0.794643
9,0.596340,0.875542,0.564451,0.609589,0.899944,0.614286,0.513514,0.956589,0.485906,0.214235,0.037134,0.223557,0.204430,0.226258,0.030769,0.872115,NaN,0.803571


In [43]:
network = Network([16, 60, 70, 2], sigmoid, sigmoid_prime, True)
errors = network.SGD(normalized_train_data, epochs=1000, alpha=0.01, classes_count=2)

AttributeError: 'str' object has no attribute 'reshape'

In [ ]:
fig = plt.gcf()
fig.set_size_inches(20, 20)

plt.subplot(3, 1, 1)
plt.plot(errors)

for class_index in range(2):
    plt.subplot(3, 1, 2 + class_index)
    draw_class(class_index, normalized_test_data, 2)

plt.show()
print(rmse(network, Network.prepare_data(normalized_test_data, 2)))